# Constrained Bayesian optimisation

In this example, NUBO is used to maximise a function where the input space is bounded and constrained. The whole process is not to different from the unconstrained case. We only need to choose a different optimiser that allows the use of constraints when maximising the acquisition function `UpperConfidenceBound`. NUBO uses the `slsqp` optimiser that can be provided with a dictionary or a tuple of dictionaries that specify one or multiple constraints. We specify two constraints to showcase the two different options: equality constraints and inequality constraints. Equality constraints require the constraint to be 0 while the result is non-negative for inequality constraints. Our first constraint `{'type': 'ineq', 'fun': lambda x: 0.5 - x[0] - x[1]}` is an inequality constraint and requires the sum of the first two inputs to be smaller or equal to 0.5. The second constraint `{'type': 'eq', 'fun': lambda x: 1.2442 - x[3] - x[4] - x[5]}` is an equality constraint specifying that the sum of the last three inputs needs to be equal to 1.2442. The `Hartmann6D` synthetic test function acts as a surrogate for a black box objective funtion, such as an experiment or a simulation. The optimisation loop is run for 40 iterations and finds a solution close the true optimum of -3.3224. Important: Generating initial input points with a Latin hypercube might not work for real problems as they will not consider the constraints but only the bounds. In these situations, other methods or selecting initial points by hand might be preferable. The purpose of this example is solely the demonstration of how NUBO handles constraints and constrained optimisation.

In [1]:
import torch
from nubo.acquisition import ExpectedImprovement, UpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import slsqp
from nubo.test_functions import Hartmann6D
from nubo.utils import gen_inputs
from gpytorch.likelihoods import GaussianLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
x_train = gen_inputs(num_points=dims*5,
                     num_dims=dims,
                     bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 40

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, lr=0.1, steps=200)

    # specify acquisition function
    # acq = ExpectedImprovement(gp=gp, y_best=torch.max(y_train))
    acq = UpperConfidenceBound(gp=gp, beta=1.96**2)

    # define constraints
    cons = ({'type': 'ineq', 'fun': lambda x: 0.5 - x[0] - x[1]},
            {'type': 'eq', 'fun': lambda x: 1.2442 - x[3] - x[4] - x[5]})
    
    # optimise acquisition function
    x_new, _ = slsqp(func=acq, bounds=bounds, constraints=cons, num_starts=5)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if y_new > torch.max(y_train[:-1]):
        print(f"New best at evaluation {len(y_train)}: \t Inputs: {x_new.numpy().reshape(dims).round(4)}, \t Outputs: {-y_new.numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {-float(y_train[best_iter]):.4f}")


New best at evaluation 37: 	 Inputs: [0.4483 0.0517 0.5663 0.1706 0.2303 0.8434], 	 Outputs: [-1.6074]
New best at evaluation 40: 	 Inputs: [0.5    0.     0.5532 0.2166 0.2412 0.7863], 	 Outputs: [-1.7706]
New best at evaluation 41: 	 Inputs: [0.5    0.     0.5418 0.2447 0.2495 0.75  ], 	 Outputs: [-1.9743]
New best at evaluation 42: 	 Inputs: [0.     0.0682 0.4714 0.245  0.2697 0.7296], 	 Outputs: [-2.5711]
New best at evaluation 43: 	 Inputs: [0.     0.15   0.4106 0.2629 0.2785 0.7028], 	 Outputs: [-2.7138]
New best at evaluation 45: 	 Inputs: [0.     0.1429 0.4903 0.2798 0.2942 0.6702], 	 Outputs: [-2.8218]
New best at evaluation 47: 	 Inputs: [0.1973 0.1472 0.4738 0.2753 0.313  0.6559], 	 Outputs: [-3.3218]
Evaluation: 47 	 Solution: -3.3218
